<a href="https://colab.research.google.com/github/Elwing-Chou/tibame0410/blob/main/tibame_0428_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request as req
import bs4 as bs
url = "https://24h.pchome.com.tw/search/?q=%E5%90%B9%E9%A2%A8%E6%A9%9F"
resp = req.urlopen(url)
html = bs.BeautifulSoup(resp.read())
links = html.find_all("a", {"class":"c-prodInfoV2__link"})
for link in links:
    link_href = "https://24h.pchome.com.tw" + link["href"]
    print(link_href)
    # enter
    p_resp = req.urlopen(link_href)
    p_html = bs.BeautifulSoup(p_resp.read())
    # 產品品牌
    p_brand = p_html.find("span", {"class":"o-prodMainName__colorSecondary"})
    p_brand_text = p_brand.text.strip()
    # 產品名稱
    p_name = p_html.find("h1", {"class":"o-prodMainName__grayDarkest"})
    p_name_text = p_name.text.strip()
    # 滿額贈
    p_labels = p_html.find_all("div", {"class":"c-label__rectangle"})
    p_full_prize = "N"
    for p_label in p_labels:
        if "滿額贈" in p_label.text:
            p_full_prize = "Y"
            break
    # 評分
    p_rating = p_html.find("div", {"class":"c-ratingIcon__textNumber"})
    if p_rating == None:
        p_rating_text = "-"
    else:
        p_rating_text = p_rating.text.strip()
    print(p_rating_text)
    print(p_brand_text)
    print(p_name_text)
    print("滿額贈:", p_full_prize)

https://24h.pchome.com.tw/prod/DMAH47-A900H319O
-
sOlac
SD-860高速智能溫控專業負離子吹風機
滿額贈: N
https://24h.pchome.com.tw/prod/DDDU04-1900I79JH
-
AVEDA
蘊活淨瑕養髮組
滿額贈: Y
https://24h.pchome.com.tw/prod/DMAHE2-1900IAF6I
-
acerpure 宏碁
Acerpure beauty 雙效負離子吹風機 HD364-10W
滿額贈: N
https://24h.pchome.com.tw/prod/DDDU04-A900HHN7T
4.5
AVEDA
蘊活煥欣洗髮精 1000ml
滿額贈: Y
https://24h.pchome.com.tw/prod/DDBXFA-1900IFCES
-
AVEDA
蘊活純香養護組
滿額贈: Y
https://24h.pchome.com.tw/prod/DMAHBY-A900I0K1F
4.9
Xiaomi 小米
負離子高速吹風機
滿額贈: N
https://24h.pchome.com.tw/prod/DMAHCW-A900IGQBP
5.0
KINYO
全球電壓無刷吹風機 KH-9660
滿額贈: N
https://24h.pchome.com.tw/prod/DMAH4C-A900I7YVV
4.9
Panasonic 國際牌
高速馬達吹風機 EH-NE6M-C 焦糖奶茶色
滿額贈: Y
https://24h.pchome.com.tw/prod/DMAH7C-A900HQBQH
4.9
TESCOM
專業負離子吹風機 TD760TW 雲朵白
滿額贈: N
https://24h.pchome.com.tw/prod/DMAH4C-A900I7YVH
5.0
Panasonic 國際牌
高速馬達吹風機 EH-NE6M-A 寶寶粉藍色
滿額贈: Y
https://24h.pchome.com.tw/prod/DMAH02-A900FUEMV
4.9
Panasonic 國際牌
高滲透奈米水離子吹風機EH-NA0J
滿額贈: Y
https://24h.pchome.com.tw/prod/DMAH02-A900AN6LF
4.9
Pana

In [ ]:
# 名字
# 原價/目前售價
# 滿額贈是否有(Y/N)
# 進去以後
# 評價幾分/幾則
# 介紹
# 圖片的網址